# XGB Model

In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from members import Members
from weather import Weather
from expeds import Expeds


# Get Data
member = Members().get_data()
member = Members().clean_data(member)

weather = Weather().get_data()
weather = Weather().clean_data(weather)

exped = Expeds().get_data()
exped = Expeds().clean_data(exped)

# Drop columns
mem_to_drop = ['memb_id','year','unique_id','peak_id','residence','occupation',
'summit_claimed','summit_disputed','highpt','high_point','death','death_type',
'death_height','death_class','summit_bid','summit_term','summit_date1',
'citizenship','o2_climb','o2_descent','o2_sleep','o2_medical', 'o2_none', 
'yob', 'route1', 'ascent1', 'leader', 'deputy', 'bconly', 'nottobc', 'support', 
'hired', 'sherpa', 'tibetan']


exp_to_drop = ['year','season','route1','route2','nation','leaders',
'sponsor','success1','success2', 'ascent1','claimed','disputed',
'countries','summit_time','term_date','term_note','high_point',
'traverse','ski','parapente','o2_climb','o2_descent','o2_sleep',
'o2_medical','o2_taken','o2_unkwn','o2_used','o2_none','other_smts',
'campsites','accidents','achievment','agency','peak_name','primmem',
'summiter_deaths','summit_members','summit_hired','hired_deaths']

member.drop(columns= mem_to_drop, inplace=True)
exped.drop(columns= exp_to_drop, inplace=True)

exped['summit_date'] = pd.to_datetime(exped.summit_date, errors = 'coerce')
exped['bc_date'] = pd.to_datetime(exped.bc_date , errors = 'coerce')
exped = exped.set_index('summit_date')
weather = weather.set_index('date_time')

# Feature Engineering (1/2)
exped['sherpa_ratio'] = exped['tot_hired'] / exped['tot_members']
exped['sherpa_ratio'] = np.where(exped['sherpa_ratio'] == np.inf, 0, exped['sherpa_ratio'])

weather['pressure_past'] = weather['pressure'].rolling(window=3).mean()
weather['pressure_futur'] = weather['pressure'].shift(-2).rolling(window=3).mean()
weather['stability'] = weather['pressure_futur'] - weather['pressure_past']

# Merge DataFrames
df = exped.merge(weather, how='left', left_index=True, right_index=True)
df = df.reset_index()
df = df.rename(columns={'index' : 'summit_date'})
df = df.merge(member, on='exp_id', how = 'right')
df.drop(columns=['moonrise', 'moonset', 'sunrise', 'sunset'], inplace = True)
df = df.dropna(subset=['summit_date', 'bc_date'])

# Feature Engineering (2/2)
df['cumul_snow'] = 0

for index, row in df.iterrows():
    date1 = row['bc_date'].date()
    date2 = row['summit_date'].date()
    acc_snow = weather.loc[date1:date2, 'totalSnow_cm'].sum()
    df.loc[index, 'cumul_snow'] = acc_snow

df.drop(columns=['summit_date', 'exp_id', 'bc_date', 'term_reason'], inplace=True)

In [41]:
# Data Processing
col_num = []
col_bool =[]
col_object =[]

for col in df:
    if df[col].dtype == "float64":
        col_num.append(col)
        
    if df[col].dtype == "int64":
        col_num.append(col)
        
    if df[col].dtype == 'bool':
        col_bool.append(col)
        
    if df[col].dtype == 'object':
        col_object.append(col)
        
col_bool.remove('summit_success')

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop= 'first', handle_unknown='error'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, col_num),
        ('cat', categorical_transformer, col_object)])

clf = Pipeline(steps=[('preprocessor', preprocessor)])

X = df.drop(columns=['summit_success'])
y = df.summit_success

X_trans = clf.fit_transform(X)
X_tr = X_trans.toarray()

X_train, X_test, y_train, y_test = train_test_split(X_tr, y, test_size= 0.3, random_state= 1)

In [42]:
# Model Traning
from sklearn.metrics import classification_report
boost = XGBClassifier()
boost.fit(X_train, y_train)
ypred = boost.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.86      0.81      0.83      2290
        True       0.78      0.83      0.80      1852

    accuracy                           0.82      4142
   macro avg       0.82      0.82      0.82      4142
weighted avg       0.82      0.82      0.82      4142



In [64]:
# Best model of Jeremy
best_of_best = XGBClassifier(n_estimators = 300,
                            max_depth = 18,
                            learning_rate = 0.3,
                            gamma = 0.8,
                            min_child_weight = 4,
                            colsample_bytree = 0.3
                            )

best_of_best.fit(X_train, y_train)
ypred = best_of_best.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.84      0.82      0.83      2290
        True       0.78      0.81      0.79      1852

    accuracy                           0.81      4142
   macro avg       0.81      0.81      0.81      4142
weighted avg       0.81      0.81      0.81      4142



## Other Models

In [56]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(max_iter=500)
reg.fit(X_train, y_train)
ypred = reg.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.73      0.76      0.75      2290
        True       0.69      0.66      0.67      1852

    accuracy                           0.72      4142
   macro avg       0.71      0.71      0.71      4142
weighted avg       0.72      0.72      0.72      4142



In [58]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.84      0.81      0.82      2290
        True       0.77      0.80      0.79      1852

    accuracy                           0.81      4142
   macro avg       0.80      0.81      0.80      4142
weighted avg       0.81      0.81      0.81      4142



## RandomSearchCV

### XGB

In [51]:
%%time
from sklearn.model_selection import RandomizedSearchCV

params = {'booster' : ['gbtree', 'gblinear'],
        'min_child_weight': [0.1, 1, 5, 10, 50],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5, 10, 25, 50],
        'learning_rate': [0.0001, 0.001, 0.1, 1],
        'n_estimators': [50, 100, 250, 500],
        'reg_alpha': [0.0001, 0.001, 0.1, 1],
        'reg_lambda': [0.0001, 0.001, 0.1, 1]
        }

model = XGBClassifier()

random_search = RandomizedSearchCV(model, 
                                   cv=5,
                                   param_distributions=params,  
                                   scoring='accuracy', 
                                   n_jobs=-1,
                                   verbose=1, 
                                   random_state=1001)

random_search.fit(X_train, y_train)

Wall time: 1min 52s


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                   param_distributions={'booster': ['gbtree', 'gblinear'],
                                        'colsample_bytree': [0.6, 0.8, 1.0],
                             

In [52]:
boost

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [53]:
best_model = random_search.best_estimator_
best_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.001, max_delta_step=0, max_depth=25,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.1, reg_lambda=0.0001, scale_pos_weight=1,
              subsample=0.8, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [54]:
ypred = best_model.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.86      0.81      0.83      2290
        True       0.78      0.84      0.81      1852

    accuracy                           0.82      4142
   macro avg       0.82      0.82      0.82      4142
weighted avg       0.82      0.82      0.82      4142



### RFC

In [59]:
%%time
from sklearn.model_selection import RandomizedSearchCV

params = {'n_estimators' : [100,200,500],
        'criterion' : ['gini', 'entropy'],
        'min_samples_split': [2, 3, 5],
        'max_depth': [5, 10, 25, 50],
        'min_samples_leaf': [1, 3, 5],
        'max_features': ["sqrt", "log2"]
        }

model = RandomForestClassifier()

random_search = RandomizedSearchCV(model, 
                                   cv=5,
                                   param_distributions=params,  
                                   scoring='accuracy', 
                                   n_jobs=-1,
                                   verbose=1, 
                                   random_state=1001)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   19.5s finished


Wall time: 21.8 s


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 10, 25, 50],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 3, 5],
                                        'min_samples_split': [2, 3, 5],
                                        'n_estimators': [100, 200, 500]},
                   random_state=1001, scoring='accuracy', verbose=1)

In [60]:
best_model = random_search.best_estimator_

In [62]:
best_model

RandomForestClassifier(criterion='entropy', max_depth=25, max_features='log2',
                       min_samples_split=5, n_estimators=200)

In [61]:
ypred = best_model.predict(X_test)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

       False       0.86      0.81      0.83      2290
        True       0.78      0.83      0.80      1852

    accuracy                           0.82      4142
   macro avg       0.82      0.82      0.82      4142
weighted avg       0.82      0.82      0.82      4142



# Test xgb_model.py

In [45]:
from xgb_model import HimalXGB

model = HimalXGB().train_model()

In [7]:
y_pred = HimalXGB().predict_model(X)

In [9]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

       False       0.92      0.88      0.90      7590
        True       0.86      0.91      0.89      6214

    accuracy                           0.90     13804
   macro avg       0.89      0.90      0.89     13804
weighted avg       0.90      0.90      0.90     13804

